In [2]:
import pandas as pd
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np
import requests

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                       

In [80]:
#define city and state for search
city_state = 'New York, NY' #can be changed to any city

#lookup latitude and longitude for city to be searched
geolocator = Nominatim(user_agent="exployer")
location = geolocator.geocode(city_state)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of',city_state,'are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York, NY are 40.7127281, -74.0060152.


In [5]:
# The code was removed by Watson Studio for sharing.

In [81]:
#define search method
search_by = 'city/state' #can be 'city/state' to search by city and state names or can be 'lat/long' to search by city latitude and longitude

LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 100000 # define radius
section = 'food'
near = city_state

#dictionary to store search results to be used to construct a dataframe
restaurants = {'name':[],'category':[],'lat':[],'long':[]}

#foursquare limits search results to a maximum of 50, however the explore endpoint allows for an offset that can "page through" the results
#if we set our maximum results to 50 then making multiple searches with offset = 0,50,100,... we can page through all results even if there are more than 50
offset = 0
search = True
while search:
    
    if search_by == 'city/state':
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&limit={}&offset={}&section={}&near={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                LIMIT,
                offset,
                section,
                near)
    if search_by == 'lat/long':
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&limit={}&offset={}&section={}&ll={},{}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                LIMIT,
                offset,
                section,
                latitude,
                longitude)
    
    results = requests.get(url).json()
    if results['response']['groups'][0]['items']:
        for venue in results['response']['groups'][0]['items']:
            try:
                restaurants['name'].append(venue['venue']['name'])
                restaurants['category'].append(venue['venue']['categories'][0]['shortName'])
                restaurants['lat'].append(venue['venue']['location']['lat'])
                restaurants['long'].append(venue['venue']['location']['lng'])
            except KeyError:
                print('')
                print('missiong info')
                print('')
    
        offset = offset + 50
    else:
        search = False

In [82]:
#dataframe that reports the name of each restaurant found in the city along with its category
restaurants_df = pd.DataFrame(restaurants)
restaurants_df

,name,category,lat,long
0,Levain Bakery,Bakery,40.781513,-73.979260
1,Los Tacos No. 1,Tacos,40.757237,-73.987454
2,Los Tacos No. 1,Tacos,40.742244,-74.005961
3,Crown Shy,Restaurant,40.706187,-74.007490
4,Los Mariscos,Seafood,40.742000,-74.005890
5,Wayla,Thai,40.718291,-73.992584
6,Faicco's Italian Specialties,Sandwiches,40.731117,-74.003043
7,Levain Bakery,Bakery,40.777354,-73.955284
8,Brooklyn Bagel & Coffee Company,Bagels,40.730913,-73.993259
9,4 Charles Prime Rib,Steakhouse,40.735219,-74.000649


In [83]:
#dataframe that reports every type of restaurant in the search results and how many times that type occures
count_df = restaurants_df['category'].value_counts().to_frame().rename(columns={'category':'count'})
count_df['category'] = count_df.index
count_df.reset_index(inplace=True,drop=True)
count_df

,count,category
0,20,Italian
1,16,Bakery
2,15,American
3,15,Pizza
4,11,New American
5,10,Sandwiches
6,10,Café
7,9,Mediterranean
8,9,Thai
9,9,Seafood


In [84]:

# create map of the city using latitude and longitude values
city_map = folium.Map(location=[latitude, longitude], zoom_start=12)

#mark each category with a unique color
category_index = count_df['category'].to_dict()
category_index = {value: key for key, value in category_index.items()}
n_categories = restaurants_df['category'].nunique()
hex_color = ['#'+hex(int(color))[2:] for color in np.linspace(0,256**3,n_categories+1)]

# add markers to map
for index,(lat, lng, name, category) in enumerate(zip(restaurants_df['lat'],restaurants_df['long'],restaurants_df['name'],restaurants_df['category'])):
    label = '{}, {}'.format(name,category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=hex_color[category_index[category]],
        fill=True,
        fill_color=hex_color[category_index[category]],
        fill_opacity=1,
        parse_html=False).add_to(city_map)  
    
city_map

Types of Restaurants to Consider Opening

In [85]:
#list of types of restaurants in a city search that only have one of its type
list(count_df[count_df['count']==1]['category'])

['Austrian',
 'Turkish',
 'Gastropub',
 'BBQ',
 'Israeli',
 'Pet Café',
 'Latin American',
 'Comfort Food',
 'Empanada',
 'Indian',
 'Snacks',
 'African',
 'Moroccan',
 'Scandinavian',
 'Donuts',
 'Molecular Gastronomy',
 'Soup',
 'Hot Dogs',
 'Fried Chicken',
 'Peruvian',
 'Food Court',
 'Szechuan',
 'Udon']